<font size=5>Run Text_Matcher Algorithm</font>

In [2]:
#Troubleshooting: !jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 if you 
# get a message about the data rate limit
from matcher import Text, Matcher
import json
import pandas as pd
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 6]

In [5]:
# Replace with the format: AuthorLastName_TitleOfText (ex: Butler_GenderTrouble)
textname = 'Preciado_2013_Testo_junkie'

# Load the data. Replace articles with the path of the jsonl file you want to use - 
# should be a file of all the articles that cite the text you're looking at.
articles = '/Users/annie/Downloads/Kuhn_Hacking_2012_The_structure_of_scientific_revolutions.jsonl'
with open(articles) as f: 
    rawArticles = f.readlines()

# Parse the data. 
data = [json.loads(line) for line in rawArticles]

df = pd.DataFrame(data)
# Drop the irrelevant columns and rows with no full text.
df = df.drop(['url', 'outputFormat', 'volumeNumber', 'subTitle', 'provider', 'datePublished', \
              'pagination', 'pageEnd', 'pageStart'], axis=1)
df.dropna(subset=['fullText'], inplace=True)
df.to_json(path_or_buf=f'{textname}_processed.jsonl', orient='records', lines=True)

processedData = f'{textname}_processed.jsonl'
with open(processedData) as f:
    rawProcessedData = f.readlines()
data = [json.loads(line) for line in rawProcessedData]

# Load the text you want to find quotations from. Replace text with the text file you want to use.

text = '/Users/annie/Downloads/Kuhn_Hacking_2012_The_structure_of_scientific_revolutions_plaintext.txt'
with open(text) as f: 
    rawText = f.read()

tx = Text(rawText, textname)

In [6]:
for i, article in enumerate(data): 
    clear_output()
    print('\r', 'Matching article %s of %s' % (i, len(data)), end='')
    if 'numMatches' not in article: 
        articleText = Text(article['fullText'], article['id'])
        article['numMatches'], article['Locations in A'], article['Locations in B'] = \
        Matcher(tx, articleText).match()
        article['fullText'] = ''

 Matching article 101 of 102

In [8]:
df = pd.DataFrame(data)
df = df.drop(['fullText'], axis=1)

#Replace 'finaldata.jsonl' with whatever name you want to give this file.
textname = 'Kuhn_Hacking_2012_The_structure_of_scientific_revolutions'
df.to_json(path_or_buf=f'{textname}.jsonl', orient='records', lines=True)
